This is an analysis of Los Angeles Crime Data. 

Dataset source:
https://data.lacity.org/A-Safe-City/Crime-Data-from-2010-to-Present/y8tr-7khq

We hope to show some insights from the data including:

    -What type of crime is most/least prevalent?
    -Where was there more crime?
    -Where was there less crime?
    -Where did most batteries occured? Show
    -When did crimes occured? Show
    -Is crime increasing or decreasing in time? Show

In [1]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.plotly as py

In [2]:
csv = '/Users/nonoumasy/Downloads/data/LA_Crime_Data_from_2010_to_Present.csv'
df = pd.read_csv(csv)

In [ ]:
#clean the data
df = df[['Date Occurred','Time Occurred', 'Area Name', 'Crime Code',
         'Crime Code Description','Weapon Description', 'Address',
        'Location ']]

df.columns = df.columns.str.replace(' ', '_')  #replace blankspace with underscore

df.columns = df.columns.str.lower()  #changes column names to lowercase

In [6]:
df.head()

,date_occurred,time_occurred,area_name,crime_code,crime_code_description,weapon_description,address,location_
0,01/16/2015,2130,Van Nuys,510,VEHICLE - STOLEN,NaN,6200 FULTON AV,"(34.1814, -118.4263)"
1,01/01/2015,230,Southeast,626,INTIMATE PARTNER - SIMPLE ASSAULT,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",200 E 97TH ST,"(33.9483, -118.2717)"
2,10/01/2015,800,77th Street,805,PIMPING,NaN,FIGUEROA ST,"(33.9807, -118.2827)"
3,01/01/2015,40,Central,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),NaN,HILL ST,"(34.0591, -118.2412)"
4,01/01/2015,240,Central,626,INTIMATE PARTNER - SIMPLE ASSAULT,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",500 W 7TH ST,"(34.0467, -118.2556)"


In [36]:
#what type of crime is most prevalent?
df.crime_code_description.value_counts()[:5]

BATTERY - SIMPLE ASSAULT              172969
BURGLARY FROM VEHICLE                 146743
VEHICLE - STOLEN                      145721
BURGLARY                              135867
THEFT PLAIN - PETTY ($950 & UNDER)    135333
Name: crime_code_description, dtype: int64

In [38]:
#what type of crime is least prevalent?
df.crime_code_description.value_counts()[-5:]

TILL TAP - ATTEMPT                          3
BLOCKING DOOR INDUCTION CENTER              3
FIREARMS RESTRAINING ORDER (FIREARMS RO)    2
TRAIN WRECKING                              2
DRUNK ROLL - ATTEMPT                        1
Name: crime_code_description, dtype: int64

In [32]:
#which areas have more crime?
df.area_name.value_counts()[:5]

77th Street    131722
Southwest      122674
N Hollywood    103320
Pacific        100906
Southeast      100288
Name: area_name, dtype: int64

In [27]:
#which areas have less crime?
df.area_name.value_counts()[-5:]

Rampart       80521
West LA       80127
Wilshire      78481
Foothill      72360
Hollenbeck    70000
Name: area_name, dtype: int64